# The FF simulator
## Note that some of this code is old as shit and I'm not quite sure why it is the way it is. I keep meaning to rewrite it, but CBA.

First of all do some imports:
- BS4 does html handling
- urlib opens websites
- numpy for decent arrays
- copy, not sure, maybe depleted!?
- random for the PRNG for the league

In [13]:
from bs4 import BeautifulSoup
import urllib
import numpy as np
from copy import copy
import random

nSims = 1000000

# Now time for the nuts and bolts of the script
1. Get fixtures list from the FF site, simple scrape used in the stats site too
2. Get the current league points, basically who has what number of points
3. Decency filter, checks that the simulated league isn't fucked up
4. Simulate league, for each GW remaining assign a random winner (or draw) to each game
5. Finalise solutions, not really sure exactly how it does it, but appears to work out who's in the top four...

In [14]:
def getFitureList():
    matches = np.zeros((36,8))
    matchCounter = 0
    for page in range(1, 4):
        fixtureURL = 'http://fantasy.premierleague.com/my-leagues/2968/matches/?matches-page='+str(page)
        r = urllib.urlopen(fixtureURL).read()
        soup = BeautifulSoup(r, 'lxml')
        rows = soup.findAll('tr', {'class': 'ismResult'})
        for row in rows:
            gwNum = int(row.td.text)
            home = row.find('td', {'class', 'ismHome'})
            homeUrl = home.find('a', href=True)['href'][7:]
            homeTeam = homeUrl[:homeUrl.find('/')]
            matches[gwNum-1, (matchCounter%8)] = homeTeam
            
            away = row.find('td', {'class', 'ismAway'})
            awayUrl = away.find('a', href=True)['href'][7:]
            awayTeam = awayUrl[:awayUrl.find('/')]
            matches[gwNum-1, (matchCounter+1)%8] = awayTeam
            
            matchCounter += 2
    return matches

def getLeaguePoints():
    baseUrl = 'http://fantasy.premierleague.com/my-leagues/2968/standings/'
    r = urllib.urlopen(baseUrl).read()
    soup = BeautifulSoup(r, 'lxml').find('table')
    rows = soup.findAll('tr')
    leagueDict = {}
    for row in rows:
        team = -1
        cells = list(row.findAll('td'))
        for cell in cells:
            teamNum = cell.find('a', href=True)
            if(teamNum is not None):
                teamNum = teamNum['href'][7:]
                team = int(teamNum[:teamNum.find('/')])
                continue
            if(team > -1):
                leagueDict[team] = int(cell.text.replace(',', ''))
    soup = BeautifulSoup(r, 'lxml').findAll('h2', { "class" : "ismTableHeading" })
    gwNums = []
    for table in soup:
        gwNums.append(int(table.text.replace('Gameweek', '').strip()))
    return leagueDict, gwNums

def decencyFilter(toCheckMatches=[]):
    for i in toCheckMatches:
        if(len(i) != 8):
            print "Length not right\n"
            exit()
        for j in i:
            if(i.count(j) != 1):
                print "Too many occurances of %s", j
    return 1


def simulatedSolution(matchLineUp=[], leagueTable={}):
    newLeagueTable = copy(leagueTable)
    for i in matchLineUp:
        for j in range(0, 8, 2):
            result = random.randint(0, 400)
            if(result < 197):
                newLeagueTable[i[j]] += 3
            elif(result < 394):
                newLeagueTable[i[j+1]] += 3
            else:
                newLeagueTable[i[j]] += 1
                newLeagueTable[i[j+1]] += 1
    outNames = []
    outScores = []
    for w in sorted(newLeagueTable, key=newLeagueTable.get, reverse=True):
        outNames.append(w)
        outScores.append(newLeagueTable[w])
    return outNames, outScores


def finaliseSolutions(scores, names):
    outNames = []
    while(len(outNames) < 4):
        for i in scores:
            if(names[scores.index(i)] not in outNames):
                if(scores.count(i) <= 4-len(outNames)):
                    j = scores.index(i)
                    while(scores[j] == i):
                        outNames.append(names[j])
                        j += 1
                else:
                    indices = [j for j, x in enumerate(scores) if x == i]
                    while(len(outNames) < 4):
                        choice = random.randint(0, len(indices)-1)
                        outNames.append(names[indices[choice]])
    return outNames

# Do setup before running simulation
1. Get fixtures
2. Get the current league
3. print both these, check that it looks right... Also can adjust number GW's remaining if runing before sit update

In [16]:
matches = getFitureList()
currentLeagueTable, gwNums = getLeaguePoints()
newMatches = matches[gwNums[0]:]
print newMatches.shape
print currentLeagueTable

(4, 8)


# Now do the actual simulation

In [18]:
topFourResults = {}
for team in newMatches[0]:
    topFourResults[team] = 0
for ff in xrange(nSims):
    outNames, outScores = simulatedSolution(newMatches,currentLeagueTable)
    results = finaliseSolutions(outScores,outNames)
    for i in results:
        topFourResults[i]+=1

# Now do some magic to turn numbers into names and percents

In [19]:
playerNames = {"Martyn":12254, "Nick":22029, "Ross":202857, "Sam":50395, "Pete":8293,"Steve":33197, "Snell":156762, "Ricky":10494}
playerNames = dict((v,k) for k,v in playerNames.iteritems())
for key, value in topFourResults.iteritems():
    print '{0:<8}{1:.3f}'.format(playerNames[key], 100*float(value)/nSims)

# If I'm running before the league has updated then I have to manually add the points to the league table

In [ ]:
currentLeagueTable[22029] += 3
currentLeagueTable[50395] += 3
currentLeagueTable[33197] += 3
currentLeagueTable[10494] += 3
print currentLeagueTable